# **This tutorial is written by Chun Lin, Chien on 2020/05/02.**
# **Topic : Vehicle Routing Problem with Capacity limitation and Time window.**
# © Chun-Lin Chien (2020)
---
```
P.S : If you use this CVRP_TW python code or refer the result in your research, please cite this tutorial.
      ( e.g, python PuLP tutorial for CVRP_TW (Chien, 2020) ).
```
---

# **// Preprocess for accessing google dirve and installing PuLP module. //**

## Pre 1. Access google drive and change dir to ./Vehicle Routing Problem.

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
data_dir = "/content/drive/My Drive/Vehicle Routing Problem"
os.chdir(data_dir)

## Pre 2. Install PuLP in your COLAB VM.

In [2]:
! pip install pulp

# **// Python code. // CVRP_TW ( Multi vehicle type )**

## Step 1. Import module.

In [3]:
import pandas as pd 
from pulp import *
from math import ceil
from pandas.core.common import flatten

## Step 2. Read cost matrix and parameters from excel file and preprocess data into query format. 

### Step 2-1. Build data generator.

In [4]:
def data_generator(excel, Cost_matrix, parameter, origin_capa=18):
    '''  
    This function will return a python dictionarys which contains cost matirx for each node to each node by each vehicle and few data (i.e., cost_matrix, data).\n   
    The explaination of each function parameter is shown as following comments.
    1. excel = excel file.
    2. Cost_matrix = The sheet name which contain cost matrix.
    3. parameter = The sheet name which contain parameter such as vehicle capacity (i.e., column = VCAP) and demand (i.e., column = Q).
    4. capacity_unit_cost = The unit cost of each capacity unit, the default value is 200.
    Hint：data . keys( ) = [ ' location ', ' demand ', ' capacity ' ]
    '''
    # Read Excel file.
    dataframe = pd.ExcelFile(excel)
    sheet_1, sheet_2 = dataframe.parse(Cost_matrix), dataframe.parse(parameter)

    # Initial data dictionary.
    data = {}

    # Main parameters.
    data['location'] = list(sheet_1.columns[1:].dropna())
    data['demand'] = sheet_2['Demand'].dropna()
    data['capacity'] = list(sheet_2['Capacity'].dropna().astype('int32'))

    # Time window parameters.
    data['earliest'] = sheet_2['Earliest time'].dropna()
    data['latest'] = sheet_2['Latest time'].dropna()
    data['duration'] = sheet_2['stop_duration'].dropna()
    data['TPM'] = sheet_2['Time Per Mile'].dropna().astype('float32')

    # Cost matrix (distance or time). Each vehicle has its own cost matrix.
    cost_matrix = {}
    for city_O in range(data['location'].__len__()):
        for city_D, cost in enumerate(sheet_1.loc[city_O,data['location']]):
            for idx, capacity in enumerate(data['capacity']):
                if city_O != city_D:
                    cost_matrix.update({'{}_{}_{}'.format(str(city_O), str(city_D), str(idx)): ceil(cost*(capacity/origin_capa))})
    data['cost_matrix']=cost_matrix
    
    # Distance matirx.
    dist_matirx = {}
    for city_O in range(data['location'].__len__()):
        for city_D, cost in enumerate(sheet_1.loc[city_O,data['location']]):
            if city_O != city_D:
                dist_matirx.update({'{}_{}'.format(str(city_O), str(city_D)): int(cost)})
    data['distance'] = dist_matirx

    return data            

### Step 2-2. Call data generator.

In [5]:
excel = 'The Vehicle Routing Problem.xlsx' # Excel file name.
Cost_matrix = 'cost_matrix_2' # Sheet name of cost matrix.
parameter = 'parameter_2' # Sheet name of parameter.
#////////////////////////////////////////////////////////////////#
data  = data_generator(excel, Cost_matrix, parameter) # data.keys() = 'location', 'demand', 'capacity', 'cost_matrix'

In [6]:
vehicle_num = data['capacity'].__len__() # Vehicle number is define by the length of capacity data. 
cost_matrix = data['cost_matrix'] # Just for coding convenience...haha.

## Step 3. Initial parameters and variables.

In [7]:
# Minimum no. vehicles required, fractional and rounded.
min_vehicle = ceil( data['demand'].sum() / (sum(data['capacity'])/vehicle_num) )
print('The least of query vehicle number = {} units.'.format(min_vehicle))

# Decision variables.
x = LpVariable.dicts('x', [i for i in cost_matrix.keys()], lowBound=0, upBound=1, cat='Binary')

# Dummy variable.
#   Accumulated delivers at city k.
u = LpVariable.dicts('u', [f"{i}_{j}" for i in range(data['location'].__len__()) for j in range(vehicle_num)], lowBound=0, upBound=max(data['capacity']), cat='Integer')
#   Arrival time at city k.
t = LpVariable.dicts('t', [f"{i}" for i in range(data['location'].__len__())], lowBound=0, cat='Integer')

The least of query vehicle number = 3 units.


## Step 4. Initial optimal problem.

### Step 4-1. Problem type and Objective function.

In [8]:
# The problem type is minimized problem.
prob = LpProblem('Capacity_Vehicle_Routing_Proble', LpMinimize)

# Objective function.
prob += lpSum( x[key] * cost_matrix[key] for key in cost_matrix.keys() )

### Step 4-2. Vehicle and Capacity constraints.

In [9]:
# Set the vehicle minimal volume. 
prob += lpSum( x[i] for i in cost_matrix.keys() if (i.split('_')[0]=='0') ) >= min_vehicle

# Must send enough vehicles out of depot.
#   HINT: (1)x_0_1_0 + x_0_2_0 + ... + x_0_n_0 <= 1
for vehicle in range(vehicle_num):
    prob += lpSum( x[i] for i in cost_matrix.keys() if (i.split('_')[0]=='0') and (i.split('_')[2]==str(vehicle)) ) <= 1

# The departure vehicle must be the return vehicle.
for i in range(vehicle_num):
#   HINT: x_0_1_0 - x_1_0_0 = 0
    for O in range(len(data['location'])):
        prob += lpSum( x[f'{O}_{D}_{i}']-x[f'{D}_{O}_{i}'] for D in range(len(data['location'])) if O!=D ) == 0

# Loop through the city (For each city, except depot).
for O in range(1,len(data['location'])):

#   A vehicle must enter it.
    prob += lpSum( x[i] for i in cost_matrix.keys() if (i.split('_')[0]==f'{O}') and  
                   (data['demand'][int(O)]+data['demand'][int(i.split('_')[1])]<=data['capacity'][int(i.split('_')[2])]) ) == 1
                   
#   A vehicle must leave it after service.
    prob += lpSum( x[i] for i in cost_matrix.keys() if (i.split('_')[1]==f'{O}') and  
                   (data['demand'][int(O)]+data['demand'][int(i.split('_')[0])]<=data['capacity'][int(i.split('_')[2])]) ) == 1 

#   The accumulated delivers at city O is at least amount needed at O but can't exceed capacity.
    prob += lpSum( u[f'{O}_{i}'] for i in range(vehicle_num) ) >= data['demand'][O]
    for i in range(vehicle_num): prob += u[f'{O}_{i}'] <= data['capacity'][i]

#   If O follows D, then can bound U(O) - U(D).
    for D in range(1, len(data['location'])):
        if O!=D :
            for i in range(vehicle_num):
                Vcap = data['capacity'][i]
                demand_O = data['demand'][O]
                demand_D = data['demand'][D]
                prob += u[f'{O}_{i}'] >= (u[f'{D}_{i}'] + demand_O - Vcap) + Vcap*(x[f'{O}_{D}_{i}']+x[f'{D}_{O}_{i}']) - ((demand_O+demand_D)*x[f'{O}_{D}_{i}'])

#   If O is 1st stop, then accumulated delivers at city O <= Q(O).
    for i in range(vehicle_num):
        Vcap = data['capacity'][i]
        demand = data['demand'][O]
        current_node = x[f'0_{O}_{i}']
        prob += u[f'{O}_{i}'] <= ( Vcap - ( Vcap - demand) * current_node )

#   If O is not 1st stop, then accumulated delivers at city O >= Q(O) + all arrived at city O.
    demand = data['demand'][O]
    temp = lpSum(u[f'{O}_{i}'] for i in range(vehicle_num)) - lpSum( (x[f'{city}_{O}_{i}'] * data['demand'][city]) for city in range(1,len(data['location'])) if city != O for i in range(vehicle_num) )
    prob += temp >= demand

### Step 4-3. Time window constraints.

In [10]:
T_Max = 99999

# Loop through the city (For each city, except depot).
for O in range(1,len(data['location'])):

    # Time of arrival at O if preceding stop was I.
    for I in range(len(data['location'])):
        if O != I:
            x_agg = lpSum(x[f'{I}_{O}_{i}'] for i in range(vehicle_num))
            temp = t[f'{I}'] + ( (data['duration'][I]+data['TPM']*data['distance'][f'{I}_{O}']) * x_agg ) - ( data['latest'][I] * (1-x_agg) )
            prob += t[f'{O}'] >= temp

    # Must arrive within the [Earliest, Latest] window. They are allowed to wait in order to arrive no earlier than t(O)_(v).
    prob += t[f'{O}'] >= data['earliest'][O]
    prob += t[f'{O}'] <= data['latest'][O]
    
    # Max trip time constraint ( T_Max = 99999 ).
    prob += t[f'{O}'] + data['duration'][O] + data['TPM']*data['distance'][f'{O}_0']*lpSum( x[f'{O}_0_{i}'] for i in range(vehicle_num) ) <= T_Max

## Step 5. Some tricks for solving result.

In [12]:
%time prob.solve()

Wall time: 9.38 s


1

In [13]:
print("Operation_status = {}\n".format(LpStatus[prob.status]))
print("Optimal_value = {}\n".format(value(prob.objective)))

Operation_status = Optimal

Optimal_value = 10483.0



In [14]:
def get_next(route):
    travel = [route[0]]
    _range_ = route.__len__()-1
    for i in range(_range_):
        start = travel[-1].split("_")[1]
        temp = [ arc for arc in route if arc.split("_")[0] == start ]
        travel.append(temp[0])
    return travel

In [15]:
result = [ k for k, v in x.items() if value(v)!=None and value(v)>0 ]

route_list = []
for i in range(vehicle_num):
    route_list.append([ f"{r.split('_')[0]}_{r.split('_')[1]}" for r in result if r.split('_')[2] == str(i) ])

travel, v_list = [], []
for idx, r in enumerate(route_list):
    if len(r) != 0:
        travel.append(get_next(r))
        v_list.append(idx)

## Final Step. Display the result.

In [16]:
print("Operation_status = {}\n".format(LpStatus[prob.status]))
print("Optimal_value = {}\n".format(value(prob.objective)))
for idx, r in zip(v_list, travel):
    print( f"● Vehicle — {idx} (Total capacity={data['capacity'][idx]}) : "+' -> '.join( [ data['location'][int(i.split('_')[0])]+" ({})".format(str(data['demand'][int(i.split('_')[0])])) for i in r]
                                                                                       + [ data['location'][0]+" ({})".format(str(data['demand'][0]))]) 
                                                                                       + " →→→ Total volumn of demand = {} units.".format(sum(data['demand'][int(i.split('_')[0])] for i in r)) )

Operation_status = Optimal

Optimal_value = 10483.0

● Vehicle — 0 (Total capacity=14) : Chicago (0) -> KC (7) -> Hous (7) -> Chicago (0) →→→ Total volumn of demand = 14 units.
● Vehicle — 1 (Total capacity=18) : Chicago (0) -> Den (6) -> Anah (5) -> Frsn (3) -> Oakl (4) -> Chicago (0) →→→ Total volumn of demand = 18 units.
● Vehicle — 2 (Total capacity=18) : Chicago (0) -> LA (14) -> LA_2 (4) -> Chicago (0) →→→ Total volumn of demand = 18 units.


# **// Appendix : Result record. //**

CVRP ( Max capacity = 18 )

Duration = 427 ms

Shortest distance = 10987.0

Vehicle — 0 : Chicago (0) -> Hous (7) -> KC (7) -> Chicago (0) →→→ Total demand = 14 units.

Vehicle — 1 : Chicago (0) -> LA (18) -> Chicago (0) →→→ Total demand = 18 units.

Vehicle — 2 : Chicago (0) -> Oakl (4) -> Frsn (3) -> Anah (5) -> Den (6) -> Chicago (0) →→→ Total demand = 18 units.

CVRP ( Multi-Vehicle capacity )

Duration = 15.5 s

Shortest distance = 10483.0

● Vehicle — 0 (Total capacity=14) : Chicago (0) -> Hous (7) -> KC (7) -> Chicago (0) →→→ Total demand = 14 units.

● Vehicle — 1 (Total capacity=18) : Chicago (0) -> Oakl (4) -> Frsn (3) -> Anah (5) -> Den (6) -> Chicago (0) →→→ Total demand = 18 units.

● Vehicle — 2 (Total capacity=18) : Chicago (0) -> LA (14) -> LA_2 (4) -> Chicago (0) →→→ Total demand = 18 units.

CVRP_TW ( Multi-Vehicle capacity )

Duration = 2.62 s

Shortest distance = 10483.0

● Vehicle — 0 (Total capacity=14) : Chicago (0) -> Hous (7) -> KC (7) -> Chicago (0) →→→ Total demand = 14 units.

● Vehicle — 1 (Total capacity=18) : Chicago (0) -> Den (6) -> Anah (5) -> Frsn (3) -> Oakl (4) -> Chicago (0) →→→ Total demand = 18 units.

● Vehicle — 2 (Total capacity=18) : Chicago (0) -> LA_2 (4) -> LA (14) -> Chicago (0) →→→ Total demand = 18 units.